# DynamicAI SageMaker Demo

## Deploying the model

<div style="margin: 0.25em 0em;">First, let's create new Amazon SageMaker endpoint and deploy our model to it.</div>
<div style="margin: 0.25em 0em;">To accomplish that, we will use <code>deploy_model()</code> function which takes following parameters:</div>
<div style="margin: 0.25em 2em;"><b>model_arn</b> – Model package ARN, which you can get from AWS Marketplace.</div>
<div style="margin: 0.25em 2em;"><b>instance_type</b> – Amazon instance type to deploy to.</div>
<div style="margin: 0.25em 0em;">NOTE: Don't forget to substitute your model_arn.</div>
<div style="margin: 0.25em 0em;">NOTE 2: This operation can take few minutes.</div>

In [1]:
import dynamic_ai as dynai
model_arn = dynai.get_model_package_arn()
dynai.deploy_model(model_arn=model_arn, instance_type='ml.m5.4xlarge')

Creating model: dynamic-ai-model-1580489541
Deploying model to endpoint: dynamic-ai-endpoint-1580489541
-------------!
Success


Quick check if the system is ready to work, using <code>is_ready</code> function:

In [2]:
dynai.is_ready()

True

## Adding a message

<div style="margin: 0.25em 0em;">Everything starts with adding a message to the system. <code>add_message</code> function processes a piece of text and returns unique message_id attached to it, which we will use later.</div>
<div style="margin: 0.25em 0em;">Note: <code>add_message</code> function can take 30..60 seconds to complete, depending on the text complexity.</div>

In [3]:
dynai.add_message('Very simple test message')

'VHnDVXRc482cJZaSbsWwro'

In [4]:
dynai.add_message('Another simple test message')
dynai.add_message('And one more simple test message')
dynai.add_message('Last simple test message')

'zY7P2zMvvEdv4CFBorW-B-'

<div style="margin: 0.25em 0em;">To quickly check all the messages that were added to the system, use <code>list_messages</code> function.</div>

In [5]:
dynai.list_messages()

{'VHnDVXRc482cJZaSbsWwro': 'Very simple test message',
 'WOKP-gP3iff-AR0KWQKye-': 'Another simple test message',
 'imvLjGrhZObuGi2M1-YeDE': 'And one more simple test message',
 'zY7P2zMvvEdv4CFBorW-B-': 'Last simple test message'}

## Setting category

Now, let's set a category for our first test message:

In [6]:
dynai.set_category('VHnDVXRc482cJZaSbsWwro', 'Test')

True

...And do the same with other test messages. This way we create <code>'Test'</code> category which contains four messages:

In [7]:
dynai.set_category('WOKP-gP3iff-AR0KWQKye-', 'Test')
dynai.set_category('imvLjGrhZObuGi2M1-YeDE', 'Test')
dynai.set_category('zY7P2zMvvEdv4CFBorW-B-', 'Test')

True

<div style="margin: 0.25em 0em;">You can get a list of all messages belonging to some particular category calling <code>list_messages</code> function with <code>category</code> parameter:</div>

In [8]:
dynai.list_messages(category='Test')

{'VHnDVXRc482cJZaSbsWwro': 'Very simple test message',
 'WOKP-gP3iff-AR0KWQKye-': 'Another simple test message',
 'imvLjGrhZObuGi2M1-YeDE': 'And one more simple test message',
 'zY7P2zMvvEdv4CFBorW-B-': 'Last simple test message'}

## Simple classification example

To show how DynamicAI classification works, I will task a system to predict a category for yet another test message using <code>predict_category</code> function:

In [9]:
id = dynai.add_message('Hey, here I am - a new test message')
dynai.predict_category(id)

{'category': 'Test', 'accuracy': 66.7, 'is_approved': True}

<div style="margin: 0.25em 0em;"><code>predict_category</code> function returns a dictionary with the following keys:</div>
<div style="margin: 0.25em 2em;"><b>category</b> – predicted category, in this case <code>'Test'</code>, which is correct result.</div>
<div style="margin: 0.25em 2em;"><b>accuracy</b> – estimated accuracy of the prediction.</div>
<div style="margin: 0.25em 2em;"><b>is_approved</b> – boolean flag, True of False; True means that our system verified the prediction and found it highly reliable (up to estimated accuracy).</div>

Another example – with a message that doesn't fit in the <code>'Test'</code> category:

In [10]:
id = dynai.add_message('Hey, here I am - a completely different message')
dynai.predict_category(id)

{'category': None, 'accuracy': 0.0, 'is_approved': False}

And the result is <code>'category': None</code>, which is correct.

## Complex classification example: Rent-A-Car corpus

<div style="margin: 0.25em 0em;">Now, let's proceed to more complex example with Rent-A-Car corpus. File <code>rent-a-car-corpus-small.txt</code> contains a small set of less than 100 messages, split into the three categories:</div>
<div style="margin: 0.25em 2em;"><b>Accident</b> – Questions related to car accidents or accident notifications.</div>
<div style="margin: 0.25em 2em;"><b>Cancel</b> – Questions related to order cancellation or cancellation requests.</div>
<div style="margin: 0.25em 2em;"><b>Invoice</b> – Invoice-related questions or requests.</div>
<div style="margin: 0.25em 0em;">This small piece of code loads the file and adds the messages from it to the DynamicAI system:</div>
<div style="margin: 0.25em 0em;">NOTE: This operation will take around one hour to complete.</div>

In [11]:
with open('rent-a-car-corpus-small.txt') as f:
    lines = [line.strip() for line in f.readlines() if not line.isspace()]

for line in lines:
    parts = line.split(':', 2)
    category = parts[0]
    message = parts[1]
    message_id = dynai.add_message(message)
    dynai.set_category(message_id, category)
    print('.', end='')
    
print('done')

................................................................................................done


Let's play! Try to add some messages to the system and see what category you'll get back!

In [12]:
id = dynai.add_message('Dear Rent-A-Car company, I have lost my invoice, can you send it to me one more time?')
dynai.predict_category(id)

{'category': 'Invoice', 'accuracy': 80.0, 'is_approved': True}

In [13]:
id = dynai.add_message('Hello! My plans have changed, so can you cancel my reservation please?')
dynai.predict_category(id)

{'category': 'Cancel', 'accuracy': 75.0, 'is_approved': True}

In [14]:
id = dynai.add_message('Hello, can you please sell me a cow?')
dynai.predict_category(id)

{'category': None, 'accuracy': 0.0, 'is_approved': False}

## Shutdown endpoint and cleanup everything

Don't forget to switch the lights off when you finish :)

In [15]:
dynai.shutdown()

## The End